### Descripción

Contexto de negocio.

El cáncer de próstata es uno de los tipos de cáncer más comunes en hombres. La detección temprana es crucial para mejorar las tasas de supervivencia. La prueba de antígeno prostático específico (PSA) puede ayudar a detectar el cáncer de próstata en etapas tempranas, cuando es más tratable. Sin embargo, el PSA no es específico para el cáncer de próstata y puede estar elevado en otras condiciones como prostatitis o hiperplasia prostática benigna (HPB).

El antígeno prostático específico (PSA) es una proteína producida por células normales y malignas de la glándula prostática. La prueba del PSA mide el nivel de esta proteína en la sangre y es uno de los métodos más utilizados para el tamizaje del cáncer de próstata.

La EPS SaludPorTi, está interesado en priorizar la toma de está prueba, aumentando la demanda y detención temprana del Cáncer de Próstata.

Problema de negocio
La empresa ha decidido contratarlos para que construyan un modelo predictivo que permita estimar la probabilidad de que un usuario entre 48 y 60 años de edad presente resultados anormales de PSA.


Contexto analítico

Se espera que entrene diferentes familias de modelos predictivos de clasificación (SVC con diferentes kernels, Redes Neuronales poco profundas), precedidos por diferentes procesos de transformación (normalizaciones, imputación, feature engineering, dummificación, PCA, selección de features).


La evaluación de la calidad de los flujos de modelos predictivos se debe estimar utilizando la métrica de ROC_AUC.

Expliquen sus ideas, el por qué realizan las acciones, y comenten los resultados obtenidos; se espera mucho más que unos bloques de código.
La toma de decisiones sobre los datos se debe hacer considerando el contexto del problema y de los datos, no se puede ver todo solamente desde los ojos de los datos, sino también considerar el negocio.
Un Científico de Datos debe poder comunicar los puntos importantes de su trabajo en un lenguaje universal para todos los públicos.
Todo esto se considerará en la nota.

### Cita

@misc{fa-ii-2024-ii-flujos-de-modelos-tradicionales,
    author = {Daniel Osorio, JavierDiaz},
    title = {FA II 2024-II: flujos de modelos tradicionales},
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/fa-ii-2024-ii-flujos-de-modelos-tradicionales}
}

### 1. Librerías a básicas utilizar

In [13]:
# Importar librerías

# Manejo de analísis de datos a través de dataframes (data tabular)
import pandas as pd
# Manipulación de arreglos y análisis numérico
import numpy as np
# Visualización de datos
import seaborn as sns
# Visualización de datos
import matplotlib.pyplot as plt
%matplotlib inline
# Librería para el manejo de expresiones regulares
import re

from sklearn.base import BaseEstimator, TransformerMixin

import warnings
warnings.filterwarnings("ignore")


### 2. Lectura de datos

In [6]:
df = pd.read_parquet(r"https://github.com/alfa7g7/Fundamentos-analitica-II/raw/refs/heads/main/UNIDAD%20II/Data/df_train.parquet")
print(df.shape)
df.head()

(23494, 46)


,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Target,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
16484,12.0,3.0,3.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,58373.057143,-110021.200000,1.885690e+05
11528,6.0,1.0,1.0,2,0.5,NaN,0.0,NaN,NaN,1.0,...,0,0,0,0,0,0,0,19618.942857,-36176.133333,9.747050e+04
26630,1.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,1,398534.000000,-626628.000000,2.304723e+06
12586,6.0,0.0,2.0,2,0.0,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,1292.600000,13410.733333,5.380450e+04
11225,3.0,0.0,2.0,1,1.0,NaN,0.0,NaN,NaN,1.0,...,2,2,2,2,2,2,0,13728.714286,-27375.666667,4.134967e+04


In [8]:
df1 = pd.read_parquet(r"https://github.com/alfa7g7/Fundamentos-analitica-II/raw/refs/heads/main/UNIDAD%20II/Data/df_test.parquet")
print(df1.shape)
df1.head()

(10069, 45)


,MEDICAMENTOS,MEDICINA ESPECIALIZADA,MEDICINA GENERAL,Cant_gr_flia,Cant_riesgos_flia_mean,min_Tiempo_CP_Fliar,Cant_Fliar_CP,psa_max_gr_flia,psa_min_gr_flia,Cant_Fliar_riesgos,...,CORONARIOS,CORONARIOS_FAMILIAR,CEREBRAL,CEREBRAL_FAMILIAR,ENFERMEDAD_RENAL,ENFERMEDAD_RENAL_FAMILIAR,OTROS_ANTECEDENTES_VASCULARES,Pendiente,Intercepto,Promedio_costo
ID,,,,,,,,,,,,,,,,,,,,,
12596,2.0,4.0,2.0,2,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,5.421180e+06,-1.256617e+07,9.611937e+06
29326,4.0,0.0,1.0,1,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,-9.504343e+03,5.997787e+04,8.013800e+04
24338,0.0,0.0,1.0,2,0.000000,NaN,0.0,NaN,NaN,0.0,...,0,0,0,0,0,0,0,2.477143e+03,-5.780000e+03,1.734000e+04
11167,1.0,0.0,0.0,1,0.000000,NaN,0.0,NaN,NaN,0.0,...,1,0,1,0,1,0,0,3.597143e+03,-8.393333e+03,2.518000e+04
22074,36.0,0.0,1.0,3,0.666667,NaN,0.0,NaN,NaN,1.0,...,0,0,0,0,0,0,0,-9.042514e+03,6.309881e+05,5.993393e+05


In [28]:
#pasar a csv
#df.to_csv(r"C:\Users\alfa7\OneDrive\Documentos\ICESI\MAESTRIA CIENCIA DE DATOS\2do semestre\Fundamentos de analitica II\Unidad II\Proyecto PSA\Data\df_train.csv")
#df1.to_csv(r"C:\Users\alfa7\OneDrive\Documentos\ICESI\MAESTRIA CIENCIA DE DATOS\2do semestre\Fundamentos de analitica II\Unidad II\Proyecto PSA\Data\df_test.csv")

### 3. Análisis Explotarorio de Datos

- Limpieza de datos
- Exploración de datos
- Ingeniería de datos

### 3.1. Limpieza de datos

Para realizar una apropiada limpieza de datos se deben entender cada una de las variables presentes dentro del conjunto de datos.

Elementos a tener en cuenta:
- El significado y tipo (e.g. nominal/ordinal/intervalo/ratio) de cada una de las variables
- Identificación de valores faltantes y en caso de haberlos plantear las opciones para tratarlos
- Presencia de datos atípicos y en caso de haberlos identificar la manera de tratarlos

In [10]:
df.columns

Index(['MEDICAMENTOS', 'MEDICINA ESPECIALIZADA', 'MEDICINA GENERAL',
       'Cant_gr_flia', 'Cant_riesgos_flia_mean', 'min_Tiempo_CP_Fliar',
       'Cant_Fliar_CP', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'Cant_Fliar_riesgos', 'cantidad_serv_flia', 'Pendiente_flia',
       'Intercepto_flia', 'Promedio_costo_flia', 'ESTADO_CIVI', 'PROGRAMA',
       'estrato', 'IMC', 'AGRUPACION_SISTOLICA', 'AGRUPACION_DIASTOLICA',
       'parentesco', 'EDAD', 'RIESGOS', 'CANTIDAD_SERVICIOS',
       'PERDIDA_DE_PESO', 'TIEMPO_ULTIMA_CITA', 'conteo_dx_diferentes',
       'TIEMPO_AFILIACION', 'CANCER_MAMA_FAMILIAR', 'CANCER_OTRO_SITIO',
       'CANCER_OTRO_SITIO_FAMILIAR', 'HIPERTENSION', 'HIPERTENSION_FAMILIAR',
       'DIABETES', 'DIABETES_FAMILIAR', 'CORONARIOS', 'CORONARIOS_FAMILIAR',
       'CEREBRAL', 'CEREBRAL_FAMILIAR', 'ENFERMEDAD_RENAL',
       'ENFERMEDAD_RENAL_FAMILIAR', 'OTROS_ANTECEDENTES_VASCULARES', 'Target',
       'Pendiente', 'Intercepto', 'Promedio_costo'],
      dtype='object')

In [11]:
df1.columns

Index(['MEDICAMENTOS', 'MEDICINA ESPECIALIZADA', 'MEDICINA GENERAL',
       'Cant_gr_flia', 'Cant_riesgos_flia_mean', 'min_Tiempo_CP_Fliar',
       'Cant_Fliar_CP', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'Cant_Fliar_riesgos', 'cantidad_serv_flia', 'Pendiente_flia',
       'Intercepto_flia', 'Promedio_costo_flia', 'ESTADO_CIVI', 'PROGRAMA',
       'estrato', 'IMC', 'AGRUPACION_SISTOLICA', 'AGRUPACION_DIASTOLICA',
       'parentesco', 'EDAD', 'RIESGOS', 'CANTIDAD_SERVICIOS',
       'PERDIDA_DE_PESO', 'TIEMPO_ULTIMA_CITA', 'conteo_dx_diferentes',
       'TIEMPO_AFILIACION', 'CANCER_MAMA_FAMILIAR', 'CANCER_OTRO_SITIO',
       'CANCER_OTRO_SITIO_FAMILIAR', 'HIPERTENSION', 'HIPERTENSION_FAMILIAR',
       'DIABETES', 'DIABETES_FAMILIAR', 'CORONARIOS', 'CORONARIOS_FAMILIAR',
       'CEREBRAL', 'CEREBRAL_FAMILIAR', 'ENFERMEDAD_RENAL',
       'ENFERMEDAD_RENAL_FAMILIAR', 'OTROS_ANTECEDENTES_VASCULARES',
       'Pendiente', 'Intercepto', 'Promedio_costo'],
      dtype='object')

Formateamos los nombres de las variables

In [14]:
def format_columns(df):
    result = df.copy()
    new_cols = []
    for col in result.columns:
        new_cols.append(re.sub(r'\s+', ' ',col.strip()).replace(' ','_').lower())
    result.columns = new_cols
    return result

In [21]:
# Creamos un Transformer para renombrar columnas
class Rename_columns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        result = X.copy()
        result = format_columns(result)
        result = result.rename(columns={'estado_civi':'estado_civil'})
        return result

In [22]:
df_renamed = Rename_columns().fit_transform(df)
df1_renamed = Rename_columns().fit_transform(df1)

In [23]:
df_renamed.columns

Index(['medicamentos', 'medicina_especializada', 'medicina_general',
       'cant_gr_flia', 'cant_riesgos_flia_mean', 'min_tiempo_cp_fliar',
       'cant_fliar_cp', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'cant_fliar_riesgos', 'cantidad_serv_flia', 'pendiente_flia',
       'intercepto_flia', 'promedio_costo_flia', 'estado_civil', 'programa',
       'estrato', 'imc', 'agrupacion_sistolica', 'agrupacion_diastolica',
       'parentesco', 'edad', 'riesgos', 'cantidad_servicios',
       'perdida_de_peso', 'tiempo_ultima_cita', 'conteo_dx_diferentes',
       'tiempo_afiliacion', 'cancer_mama_familiar', 'cancer_otro_sitio',
       'cancer_otro_sitio_familiar', 'hipertension', 'hipertension_familiar',
       'diabetes', 'diabetes_familiar', 'coronarios', 'coronarios_familiar',
       'cerebral', 'cerebral_familiar', 'enfermedad_renal',
       'enfermedad_renal_familiar', 'otros_antecedentes_vasculares', 'target',
       'pendiente', 'intercepto', 'promedio_costo'],
      dtype='object')

In [24]:
df1_renamed.columns

Index(['medicamentos', 'medicina_especializada', 'medicina_general',
       'cant_gr_flia', 'cant_riesgos_flia_mean', 'min_tiempo_cp_fliar',
       'cant_fliar_cp', 'psa_max_gr_flia', 'psa_min_gr_flia',
       'cant_fliar_riesgos', 'cantidad_serv_flia', 'pendiente_flia',
       'intercepto_flia', 'promedio_costo_flia', 'estado_civil', 'programa',
       'estrato', 'imc', 'agrupacion_sistolica', 'agrupacion_diastolica',
       'parentesco', 'edad', 'riesgos', 'cantidad_servicios',
       'perdida_de_peso', 'tiempo_ultima_cita', 'conteo_dx_diferentes',
       'tiempo_afiliacion', 'cancer_mama_familiar', 'cancer_otro_sitio',
       'cancer_otro_sitio_familiar', 'hipertension', 'hipertension_familiar',
       'diabetes', 'diabetes_familiar', 'coronarios', 'coronarios_familiar',
       'cerebral', 'cerebral_familiar', 'enfermedad_renal',
       'enfermedad_renal_familiar', 'otros_antecedentes_vasculares',
       'pendiente', 'intercepto', 'promedio_costo'],
      dtype='object')